In [4]:
import pandas as pd
df = pd.read_csv('/content/SMSSpamCollection.txt', sep='\t',names=['label', 'message'])
df

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
X = list(df['message'])
X

In [ ]:
y = list(df['label'])
y

In [ ]:
y = list(pd.get_dummies(y,drop_first=True)['spam'])
y

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=42,stratify=y)

In [ ]:
!pip install transformers

In [13]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [15]:
train_encoding = tokenizer(X_train, padding=True, truncation=True)
test_encoding = tokenizer(X_test, padding=True, truncation=True)

In [17]:
type(test_encoding)

transformers.tokenization_utils_base.BatchEncoding

In [46]:
test_encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [18]:
import tensorflow as tf

In [33]:
from transformers import TFDistilBertForSequenceClassification

In [52]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    train_encoding['input_ids'],
    y_train
))
train_dataset = train_dataset.batch(8)

test_dataset = tf.data.Dataset.from_tensor_slices((
    test_encoding['input_ids'],
    y_test
))
test_dataset = test_dataset.batch(8)

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=2)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=loss_fn)

model.fit(train_dataset,epochs=2)